In [206]:
import torch
import numpy as np

In [207]:
from torch import nn

In [208]:
torch.__version__

'2.0.0+cpu'

In [209]:
import pandas as pd

In [210]:
df = pd.read_csv('Stall-Windows - Stall-3s.csv')

In [211]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [212]:
#df = df.replace([' ', '-',np.nan], 0) # There are null values
df = df.replace([' ', '-',np.nan], np.nan)

In [213]:
# Selective columns for mean calculation
columns_to_convert = ['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']
df[columns_to_convert] = df[columns_to_convert].astype(float)

# Replace np.nan with mean values for selective columns
df[columns_to_convert] = df[columns_to_convert].fillna(df[columns_to_convert].mean())

# Display the modified DataFrame
# print(df)

In [214]:
# Check which columns contain np.nan values
columns_with_nan = df.isna().any()
# Display the columns with np.nan values
# print(columns_with_nan)

In [215]:
df['Stall'].replace('Yes', 1, inplace=True)
df['Stall'].replace('No', 0, inplace=True)

In [216]:
df.columns

Index(['ID', 'Stall', 'Quality', 'Time', 'CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P'],
      dtype='object')

In [217]:
X = df[['CQI1', 'CQI2', 'CQI3', 'cSTD CQI',
       'cMajority', 'c25 P', 'c50 P', 'c75 P', 'RSRP1', 'RSRP2', 'RSRP3',
       'pMajority', 'p25 P', 'p50 P', 'p75 P', 'RSRQ1', 'RSRQ2', 'RSRQ3',
       'qMajority', 'q25 P', 'q50 P', 'q75 P', 'SNR1', 'SNR2', 'SNR3',
       'sMajority', 's25 P', 's50 P', 's75 P']].values

In [218]:
y =df['Stall'].values

In [219]:
X.shape, y.shape

((4393, 29), (4393,))

In [220]:
import numpy as np

In [223]:
from sklearn.model_selection import train_test_split

In [224]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [225]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
X.shape

(6856, 29)

In [226]:
X = torch.from_numpy(X).type(torch.float32)
y = torch.from_numpy(y).type(torch.float32)

In [227]:
X.shape

torch.Size([6856, 29])

In [228]:
X[0]

tensor([ 1.4045,  1.3930, -0.8155,  2.2628,  1.3959,  0.5140,  1.3079,  1.1926,
        -0.4065, -0.3873, -0.3841, -0.3863, -0.3566, -0.3823, -0.4172,  0.8875,
        -0.5047, -0.5005, -0.5074, -0.4656, -0.5275,  0.0740,  0.7399,  0.9711,
         0.9757,  0.9732,  0.9451,  0.9883,  0.9464])

In [229]:
y.shape

torch.Size([6856])

In [230]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42
)

X_train[:1],X_train[:1].shape, y_train[:1]

(tensor([[-0.2205, -0.2119,  0.7936,  0.5628, -0.2132,  0.0630, -0.0654,  0.2309,
          -0.0034,  0.0246, -0.1771,  0.0256, -0.1501, -0.0965, -0.1299, -2.1351,
          -2.1247, -1.5803, -2.1267, -2.0247, -2.0560, -1.8691, -1.2545, -1.1988,
          -1.1523, -1.1997, -1.1654, -1.1530, -1.1706]]),
 torch.Size([1, 29]),
 tensor([1.]))

In [231]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [232]:
# Build model with non-linear activation function
from torch import nn
class InteruptionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=29, out_features=200)
        self.layer_2 = nn.Linear(in_features=200, out_features=100)
        self.layer_3 = nn.Linear(in_features=100, out_features=1)
        self.relu = nn.ReLU() # <- add in ReLU activation function
        # Can also put sigmoid in the model
        # This would mean you don't need to use it on the predictions
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
      # Intersperse the ReLU activation function between layers
       return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_3 = InteruptionModel().to(device)
print(model_3)

InteruptionModel(
  (layer_1): Linear(in_features=29, out_features=200, bias=True)
  (layer_2): Linear(in_features=200, out_features=100, bias=True)
  (layer_3): Linear(in_features=100, out_features=1, bias=True)
  (relu): ReLU()
)


In [233]:
# Setup loss and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_3.parameters(), lr=0.0001)

In [234]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [235]:
# Fit the model
torch.manual_seed(42)
epochs = 3500

# Put all data on target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)


for epoch in range(epochs):
    # 1. Forward pass
    y_logits = model_3(X_train).squeeze()

    y_pred = torch.round(torch.sigmoid(y_logits)) # logits -> prediction probabilities -> prediction labels

    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) # BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_3.eval()
    with torch.no_grad():
      # 1. Forward pass
        test_logits = model_3(X_test).squeeze()
        #print(test_logits.shape)
        test_pred = torch.round(torch.sigmoid(test_logits)) # logits -> prediction probabilities -> prediction labels
        # 2. Calcuate loss and accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)


    # Print out what's happening
    if epoch % 500 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.69241, Accuracy: 51.04% | Test Loss: 0.69078, Test Accuracy: 52.19%
Epoch: 500 | Loss: 0.47805, Accuracy: 77.17% | Test Loss: 0.47283, Test Accuracy: 78.94%
Epoch: 1000 | Loss: 0.37200, Accuracy: 84.77% | Test Loss: 0.40628, Test Accuracy: 82.36%
Epoch: 1500 | Loss: 0.27046, Accuracy: 89.72% | Test Loss: 0.35737, Test Accuracy: 84.62%
Epoch: 2000 | Loss: 0.18871, Accuracy: 93.65% | Test Loss: 0.33370, Test Accuracy: 85.64%
Epoch: 2500 | Loss: 0.13691, Accuracy: 95.46% | Test Loss: 0.33845, Test Accuracy: 87.03%
Epoch: 3000 | Loss: 0.10266, Accuracy: 97.05% | Test Loss: 0.35601, Test Accuracy: 88.56%


In [236]:
model_3.eval()
with torch.no_grad():
     y_preds = torch.round(torch.sigmoid(model_3(X_test))).squeeze()

In [237]:
y_preds.shape,y_test.shape

(torch.Size([1372]), torch.Size([1372]))

In [238]:
predictions = y_preds.cpu().numpy() #if it is cuda, then this, otherwise y_pred.numpy()
true_labels = y_test.cpu().numpy()

In [239]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score,fbeta_score

print("=== Confusion Matrix ===")
print(confusion_matrix(true_labels, predictions))
print('\n')


print("=== Score ===")
accuracy = accuracy_score(true_labels, predictions)
print('Accuracy: %f' % accuracy)

precision = precision_score(true_labels,  predictions, average='weighted')
print('Precision: %f' % precision)
recall = recall_score(true_labels, predictions, average='weighted')
print('Recall: %f' % recall)

microf1 = f1_score(true_labels, predictions, average='micro')
print('Micro F1 score: %f' % microf1)
macrof1 = f1_score(true_labels, predictions, average='macro')
print('Macro F1 score: %f' % macrof1)

=== Confusion Matrix ===
[[592  95]
 [ 59 626]]


=== Score ===
Accuracy: 0.887755
Precision: 0.888834
Recall: 0.887755
Micro F1 score: 0.887755
Macro F1 score: 0.887686


In [240]:
target_names = ['No-Stall', 'Stall']
# Print precision-recall report
print(classification_report(true_labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

    No-Stall       0.91      0.86      0.88       687
       Stall       0.87      0.91      0.89       685

    accuracy                           0.89      1372
   macro avg       0.89      0.89      0.89      1372
weighted avg       0.89      0.89      0.89      1372



In [ ]:
#Done